# Some data cleaning
So I was able to create the dataset I desired with GM data. I still would like to add coaching data, but with what I have I can start to run some analysis. I've been lucky enough to bounce analysis ideas off of a couple of friends (CJ and Adam thank you) and who have been very helpful running a little preliminary analysis to see if what comes out makes any sense and checking for errors in the data. This led to this hopefully short post on some data cleaning and getting it ready for analysis.

## Inputting Trades
In my last post I ended by stating that I wanted to input draftee trade deals such that the GM that traded for the draftee would be credited with the pick. I wanted to do this for two reasons: (1) I don't believe that a team like Charlotte should be credited with a pick like Kobe Bryant when they traded him away before seeing his true value (or conversely if he had been terrible, then the Lakers should have been 'blamed' for that pick) and (2) teams sometimes draft a player with a trade already in mind and the pick is made on behalf of another team. In trying to put these trade deals in, I was having to do it by hand, as all the information was not contained in one site. Unfortunately, this had many drawbacks, namely that I couldn't verify whether I was getting all the trades. It also seems that BR had put in some of these trade deals, but not others. Whether BR has inputted them or not, I'm not sure what the criteria are (perhaps the trade had to happen prior to a certain point on draft day?). Regardless, since I couldn't easily verify trades and that I realized if I was instituting these deals then really I should be assigning to the player the team that he signed with his rookie season (which might also be different from the team that drafted him if the drafting team waives their rights to sign him) all led me to abandon this editing of the data. Though I still consider this to be the more appropriate data, for my purposes of learning Python and demonstrating that I know how to analyze data, I will simply use the BR data as it came. 
I am stating all this up front to also demonstrate that I understand issues that arise with data and am able to address them. Now, onto the cleaning and manipulating.

In [1]:
# Standard imports 
import pandas as pd
from pandas import Series,DataFrame,read_html
import numpy as np

In [2]:
# Loading up the dataframe
df = pd.read_csv('NBA_Data/1976_to_2015_Draftees.csv')

For use in later analysis on how well a GM drafted, I believe it would be useful to know if the player attended college. Ideally I would institute the total number of years a player played in college, but for this analysis I'm trying to go a little more quickly to get to the analysis and don't have as much time to find all the college basketball info. 
The reason I'm adding whether a player went to college as a dummy variable is that a player with college experience provides more information to a GM when drafting. You would likely expect that players who have played in college are better known quantities and thus GM's should have less variance in drafting such players. Unfortunately, this variable only captures if a player played in college, so it misses those that played overseas before coming to the US. Players coming from foreign leagues also might offer more data insight prior to draft, but in this analysis will be grouped with those coming right out of high school which we would expect to close the gap some between the variance in player quality when drafted between college and non-college drafted players. Ultimately this is not a significant problem though as it should simply lead to underestimating the impact of having gone to college.

In [3]:
# Creating a dummy variable for if a player went to college or not
df['attend_college'] = 1
df.loc[df['College'] == '0', 'attend_college'] = 0

Wanted to show that I know how to make a function, even if it's a really simple one!

In [4]:
# Alternative method for creating the dummy variable 
def dummyVar(df):
    if df['College'] == '0':
        return 0
    else:
        return 1

df['attend_college2'] = df.apply(dummyVar, axis=1)

In [5]:
# Demonstrating that those players who did not go to college also got a zero in the 'attend_college' column
df[df['attend_college'] == 0].head()

,Unnamed: 0,Player,All_NBA,All-Star,Draft_Yr,Pk,Team,College,Yrs,Games,...,TRB per game,Assits per Game,Win Share,WS_per_game,BPM,VORP,Executive,Tenure,attend_college,attend_college2
1,1,Kobe Bryant,15.0,18.0,1996.0,13.0,NOH,0,20.0,1346.0,...,5.2,4.7,172.7,0.170,3.9,72.1,Bob Bass,3292 days 00:00:00.000000000,0,0
19,19,LeBron James,12.0,12.0,2003.0,1.0,CLE,0,13.0,987.0,...,7.2,6.9,192.5,0.240,9.2,108.6,Jim Paxson,2151 days 00:00:00.000000000,0,0
21,21,Dirk Nowitzki,12.0,13.0,1998.0,9.0,MIL,0,18.0,1340.0,...,7.9,2.5,198.8,0.202,3.5,65.7,Bob Weinhauer,836 days 00:00:00.000000000,0,0
84,84,Kevin Garnett,9.0,15.0,1995.0,5.0,MIN,0,21.0,1462.0,...,10.0,3.7,191.4,0.182,5.4,94.0,Kevin McHale,4960 days 00:00:00.000000000,0,0
94,94,Dwight Howard,8.0,8.0,2004.0,1.0,ORL,0,12.0,880.0,...,12.7,1.5,113.2,0.175,2.7,36.5,John Weisbrod,437 days 00:00:00.000000000,0,0


In [6]:
# Demonstrating that those players who did  go to college also got a one in the 'attend_college' column
df[df['attend_college'] == 1].head()

,Unnamed: 0,Player,All_NBA,All-Star,Draft_Yr,Pk,Team,College,Yrs,Games,...,TRB per game,Assits per Game,Win Share,WS_per_game,BPM,VORP,Executive,Tenure,attend_college,attend_college2
0,0,Tim Duncan,15.0,15.0,1997.0,1.0,SAS,Wake Forest University,19.0,1392.0,...,10.8,3.0,206.4,0.209,5.5,89.3,Gregg Popovich,2953 days 00:00:00.000000000,1,1
2,2,Tony Delk,0.0,0.0,1996.0,16.0,NOH,University of Kentucky,10.0,545.0,...,2.5,1.9,19.5,0.080,-1.3,2.1,Bob Bass,3292 days 00:00:00.000000000,1,1
3,3,Malik Rose,0.0,0.0,1996.0,44.0,NOH,Drexel University,13.0,813.0,...,4.1,0.8,26.5,0.095,-1.4,1.9,Bob Bass,3292 days 00:00:00.000000000,1,1
4,4,Shaquille O'Neal,14.0,15.0,1992.0,1.0,ORL,Louisiana State University,19.0,1207.0,...,10.9,2.5,181.7,0.208,5.0,74.0,Pat Williams,3406 days 00:00:00.000000000,1,1
5,5,Karl Malone,14.0,14.0,1985.0,13.0,UTA,Louisiana Tech University,19.0,1476.0,...,10.1,3.6,234.6,0.205,5.4,102.5,Frank Layden,2959 days 00:00:00.000000000,1,1


In [7]:
# This bit of code creates a unique executive id and assigns it to each row of the executive
# While this isn't necessary for my analysis, it may come in handy later and definitely would be 
# necessary were I dealing with a larger dataset where speed of analysis was an issue.

keys = pd.unique(df.Executive.ravel()) # returns an array of each unique executive
d = dict(zip(keys, range(1,len(keys)))) # create a dictionary of unique id's for each executive
df['Exec_id'] = df.Executive.map(d) # mapping executive id's to the executive in the dataframe

df = df.sort_values(by=['Executive', 'Exec_id']) 
df.ffill() # giving each instance of an executive the proper exec_id
df.head()

,Unnamed: 0,Player,All_NBA,All-Star,Draft_Yr,Pk,Team,College,Yrs,Games,...,Assits per Game,Win Share,WS_per_game,BPM,VORP,Executive,Tenure,attend_college,attend_college2,Exec_id
199,199,Chris Mullin,4.0,5.0,1985.0,7.0,GSW,St. John's University,16.0,986.0,...,3.5,93.1,0.139,2.8,38.7,Al Attles,3641 days 00:00:00.000000000,1,1,33.0
200,200,Bobby Lee Hurt,0.0,0.0,1985.0,42.0,GSW,University of Alabama,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,Al Attles,3641 days 00:00:00.000000000,1,1,33.0
201,201,Brad Wright,0.0,0.0,1985.0,49.0,GSW,"University of California, Los Angeles",2.0,16.0,...,0.1,-0.1,-0.045,-10.3,-0.3,Al Attles,3641 days 00:00:00.000000000,1,1,33.0
202,202,Luster Goodwin,0.0,0.0,1985.0,71.0,GSW,University of Texas at El Paso,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,Al Attles,3641 days 00:00:00.000000000,1,1,33.0
203,203,Greg Cavener,0.0,0.0,1985.0,95.0,GSW,University of Missouri,0.0,0.0,...,0.0,0.0,0.000,0.0,0.0,Al Attles,3641 days 00:00:00.000000000,1,1,33.0


## GM Drafting Experience
I next wanted to create a column denoting for each pick how many years of experience the GM had at drafting at that point. So for a GM's first year of drafting this column would be one, in his second year this would be two, and so on. Though this is likely to be heavily correlated with draft pick quality (you'd expect GM's who drafted well get to keep on drafting), I think it would be good to control for this as we would also expect GM's with more experience in drafting to draft better in later years. This seems particularly interesting for GM's like Larry Bird who have come in and out of the position. Is Larry Bird as good at drafting coming back to the job say four years after his last draft, or is it likely that a GM loses touch with the game to the extent that what one looks for in a player has changed?

To put in this column, at first I thought to write a function again, similar to the one above. I almost immediately realized that this would be more difficult than the Python method 'Categorical' and that my function variable 'count' was not going to work. I left it here anyway to show a little of my thought process and that I still screw up frequently.

In [8]:
# failed function
def draftExp2(df):
    count = 1
    if df['Exec_draft_exp2'] == 0:
        count = count + 1
        return count
    elif df['Exec_draft_exp2'] != 0:
        return 1
    
df['Exec_draft_exp2'] = df.apply(draftExp2, axis=1)
df

KeyError: ('Exec_draft_exp2', u'occurred at index 199')

In [8]:
# Very easy method to use from the pandas library
df['Exec_draft_exp'] = pd.Categorical(pd.factorize(df.Draft_Yr)[0] + 1)

In [9]:
# Demonstrating the new Exec_draft_exp column!
df[df['Executive'] == 'Al Attles'].head(25)

,Unnamed: 0,Player,All_NBA,All-Star,Draft_Yr,Pk,Team,College,Yrs,Games,...,Win Share,WS_per_game,BPM,VORP,Executive,Tenure,attend_college,attend_college2,Exec_id,Exec_draft_exp
199,199,Chris Mullin,4.0,5.0,1985.0,7.0,GSW,St. John's University,16.0,986.0,...,93.1,0.139,2.8,38.7,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,1
200,200,Bobby Lee Hurt,0.0,0.0,1985.0,42.0,GSW,University of Alabama,0.0,0.0,...,0.0,0.000,0.0,0.0,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,1
201,201,Brad Wright,0.0,0.0,1985.0,49.0,GSW,"University of California, Los Angeles",2.0,16.0,...,-0.1,-0.045,-10.3,-0.3,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,1
202,202,Luster Goodwin,0.0,0.0,1985.0,71.0,GSW,University of Texas at El Paso,0.0,0.0,...,0.0,0.000,0.0,0.0,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,1
203,203,Greg Cavener,0.0,0.0,1985.0,95.0,GSW,University of Missouri,0.0,0.0,...,0.0,0.000,0.0,0.0,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,1
204,204,Gerald Crosby,0.0,0.0,1985.0,117.0,GSW,University of Georgia,0.0,0.0,...,0.0,0.000,0.0,0.0,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,1
205,205,Eric Boyd,0.0,0.0,1985.0,141.0,GSW,North Carolina Agricultural and Technical Stat...,0.0,0.0,...,0.0,0.000,0.0,0.0,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,1
331,331,Robert Parish,2.0,9.0,1976.0,8.0,GSW,Centenary College of Louisiana,21.0,1611.0,...,147.0,0.154,1.6,41.5,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,2
332,332,Sonny Parker,0.0,0.0,1976.0,17.0,GSW,Texas A&M University,6.0,452.0,...,26.9,0.118,2.2,11.5,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,2
333,333,Marshall Rogers,0.0,0.0,1976.0,34.0,GSW,University of Texas-Pan American,1.0,26.0,...,-0.2,-0.043,-9.9,-0.4,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,2


## Duplicated players
Now comes some quality checks on potential duplicates. I found a number of them and there are two types of duplicates.
Type 1: Players who were drafted, but then decided to not enter the NBA and went back to college (see Mark Eaton) and then were later drafted again. These players have exactly the same stats, but different draft years, pick number, team, and GM's. I will drop the earlier entry, though I think an argument can be made that these players should simply be left in the dataframe. I am choosing to drop them because playing more basketball before entering the NBA provided the later GM with more information to assess player ability and thus that GM with greater information should have made a better choice than the earlier GM. But the earlier GM should also not get 'credit' for a player that took more time to develop in college and potentially entered the NBA at the 'right time' for himself.
Type 2: Players who have the same name as another player. There are more of these than one would think and they present a challenge as when I merged the player draft data with the player accomplishment data, both players picked up the player accomplishment data (all-star appearances and all-NBA teams). I deal with these later.

In [10]:
# Type 1 players, there are 24 in all
df[df.duplicated(['Player','Yrs','Games','College'])]

,Unnamed: 0,Player,All_NBA,All-Star,Draft_Yr,Pk,Team,College,Yrs,Games,...,Win Share,WS_per_game,BPM,VORP,Executive,Tenure,attend_college,attend_college2,Exec_id,Exec_draft_exp
2228,2228,Bobby Lee Hurt,0.0,0.0,1986.0,121.0,GSW,University of Alabama,0.0,0.0,...,0.0,0.000,0.0,0.0,Al Attles,3641 days 00:00:00.000000000,1,1,33.0,11
763,763,Bruce Parkinson,0.0,0.0,1977.0,105.0,WAS,Purdue University,0.0,0.0,...,0.0,0.000,0.0,0.0,Bob Ferry,6208 days 00:00:00.000000000,1,1,73.0,3
1147,1147,Tony Smith,0.0,0.0,1979.0,115.0,NJN,"University of Nevada, Las Vegas",0.0,0.0,...,0.0,0.000,0.0,0.0,Charles Theokas,1100 days 00:00:00.000000000,1,1,89.0,5
158,158,Larry Spicer,0.0,0.0,1979.0,156.0,MIL,University of Alabama at Birmingham,0.0,0.0,...,0.0,0.000,0.0,0.0,Don Nelson,3694 days 00:00:00.000000000,1,1,26.0,5
865,865,Kim Anderson,0.0,0.0,1977.0,28.0,POR,University of Missouri,1.0,21.0,...,-0.4,-0.078,-8.3,-0.4,Harry Glickman,4155 days 00:00:00.000000000,1,1,65.0,3
653,653,John Irving,0.0,0.0,1976.0,150.0,PHO,Hofstra University,0.0,0.0,...,0.0,0.000,0.0,0.0,Jerry Colangelo,9550 days 00:00:00.000000000,1,1,42.0,2
1142,1142,Mark Eaton,0.0,0.0,1979.0,107.0,PHO,"University of California, Los Angeles",11.0,875.0,...,44.8,0.085,1.5,22.4,Jerry Colangelo,9550 days 00:00:00.000000000,1,1,42.0,5
599,599,Lars Hansen,0.0,0.0,1976.0,37.0,CHI,University of Washington,1.0,15.0,...,0.6,0.149,-1.1,0.0,Jerry Krause,90 days 00:00:00.000000000,1,1,57.0,2
1250,1250,Steve Schall,0.0,0.0,1979.0,106.0,SAS,University of Arkansas,0.0,0.0,...,0.0,0.000,0.0,0.0,John Begzos,2030 days 00:00:00.000000000,1,1,85.0,5
1158,1158,Chad Nelson,0.0,0.0,1979.0,196.0,ATL,Drake University,0.0,0.0,...,0.0,0.000,0.0,0.0,Michael Gearon,653 days 00:00:00.000000000,1,1,88.0,5


In [11]:
# Dropping the first instance of the Type 1 duplicates
df.drop_duplicates(subset = ['Player', 'College', 'Yrs', 'Games'], keep = 'last', inplace=True)

In [12]:
# Showing that there are no more Type 1 duplicates
df[df.duplicated(['Player','Yrs','Games','College'])]

,Unnamed: 0,Player,All_NBA,All-Star,Draft_Yr,Pk,Team,College,Yrs,Games,...,Win Share,WS_per_game,BPM,VORP,Executive,Tenure,attend_college,attend_college2,Exec_id,Exec_draft_exp


## Check to make sure it took the later entry
Aryvdas Sabonis was a type 1 duplicate. I am loading his information here now to show that I did in fact drop the earlier entry and kept the 'duplicate'.

In [13]:
df[df['Player'] == 'Arvydas Sabonis']

,Unnamed: 0,Player,All_NBA,All-Star,Draft_Yr,Pk,Team,College,Yrs,Games,...,Win Share,WS_per_game,BPM,VORP,Executive,Tenure,attend_college,attend_college2,Exec_id,Exec_draft_exp
2249,2249,Arvydas Sabonis,0.0,0.0,1985.0,77.0,ATL,0,7.0,470.0,...,47.3,0.2,3.8,16.6,Stan Kasten,3740 days 00:00:00.000000000,0,0,63.0,1


In [14]:
# The Type 2 duplicates, there are 56 in all
df[df.duplicated(['Player'])].head()

,Unnamed: 0,Player,All_NBA,All-Star,Draft_Yr,Pk,Team,College,Yrs,Games,...,Win Share,WS_per_game,BPM,VORP,Executive,Tenure,attend_college,attend_college2,Exec_id,Exec_draft_exp
752,752,Harry Davis,0.0,0.0,1978.0,33.0,CLE,Florida State University,2.0,44.0,...,0.3,0.033,-5.4,-0.4,Bill Fitch,2218 days 00:00:00.000000000,1,1,74.0,4
637,637,Ron Davis,0.0,0.0,1976.0,70.0,ATL,Washington State University,3.0,78.0,...,0.1,0.007,-5.0,-0.7,Bud Seretean,560 days 00:00:00.000000000,1,1,72.0,2
1029,1029,Mike Phillips,0.0,0.0,1978.0,45.0,NJN,University of Kentucky,0.0,0.0,...,0.0,0.000,0.0,0.0,Charles Theokas,1100 days 00:00:00.000000000,1,1,89.0,4
1326,1326,Ken Jones,0.0,0.0,1980.0,109.0,MIL,Virginia Commonwealth University,0.0,0.0,...,0.0,0.000,0.0,0.0,Don Nelson,3694 days 00:00:00.000000000,1,1,26.0,6
2473,2473,Michael Anderson,0.0,0.0,1988.0,73.0,IND,Drexel University,1.0,36.0,...,0.2,0.010,-3.4,-0.3,Donnie Walsh,6291 days 00:00:00.000000000,1,1,38.0,13


## Only a small problem for the merged player accomplishment data though
In looking through thte Type 2 duplicates I can look at just the All-NBA column to see where a mistake with merging the player accomplishment data might have arose. Since there are only 55 Type 2 duplicates, this is fairly quick and I found only 4 where this occurred (Patrick Ewing, Glen Rice, Tim Hardaway, Larry Johnson). Given how few there are I will just fix this by hand in the final dataframe.

In [15]:
# I hate unneeded columns, so here I am looking at the columns and seeing which can be dropped
df.columns

Index([u'Unnamed: 0', u'Player', u'All_NBA', u'All-Star', u'Draft_Yr', u'Pk',
       u'Team', u'College', u'Yrs', u'Games', u'Minutes Played', u'PTS',
       u'TRB', u'AST', u'FG_Percentage', u'TP_Percentage', u'FT_Percentage',
       u'Minutes per Game', u'Points per Game', u'TRB per game',
       u'Assits per Game', u'Win Share', u'WS_per_game', u'BPM', u'VORP',
       u'Executive', u'Tenure', u'attend_college', u'attend_college2',
       u'Exec_id', u'Exec_draft_exp'],
      dtype='object')

In [16]:
# Dropping columns
df.drop(df.columns[[0,-3]], inplace=True, axis=1)

In [17]:
cd NBA_Data

/Users/rorypulvino/Dropbox (Personal)/Python/blog/content/NBA_Data


In [18]:
df.to_csv('1976_to_2015_Draftees_edit.csv')

# Finally, next some analysis
So I've cleaned the data and added some columns I'll use later for analysis, but I'm really anxious to get to analyzing. My next post I'll actually be doing some stuff to pull out insights from the data. I'll actually be doing this with some help from a friend and I'll be using R for the analysis. I've been playing around with learning R recently and it seems easier and more friendly for actual analysis work. The friend I will be collaborating with also already poked around the data using R, so we will be building off some of his preliminary work.
I will also be trying to carry out the same analysis in Python, but I will be putting those only in a notebook that I share on github and not as a blog post.